In [1]:
!nvcc --version

zsh:1: command not found: nvcc


In [2]:
!pip install git+https://github.com/afnan47/cuda.git
%load_ext nvcc_plugin

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/afnan47/cuda.git to /tmp/pip-req-build-vsbv120z
  Running command git clone --filter=blob:none --quiet https://github.com/afnan47/cuda.git /tmp/pip-req-build-vsbv120z
  Resolved https://github.com/afnan47/cuda.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4306 sha256=ce68a175d1a867ca8e4be0eecb0f59bc0070324c30d17b1932c900cf4d374939
  Stored in directory: /tmp/pip-ephem-wheel-cache-aqifizfo/wheels/aa/f3/44/e10c1d226ec561d971fcd4b0463f6bff08602afa928a3e7bc7
Successfully built NVCCPlugin
created output directory at /home/sumedh/zNotes/Engineering_Notes/BE/HPC/HPC (High Performance Computing)/Practical 4 (CUDA)/src
Out bin /home/sumedh/zNotes/Engineering_Notes/BE/HPC/HPC (High Performance Computing)/Practical 4 (CUDA)/result.out


# **Vector Addition**

In [3]:
%%cu
#include "stdio.h"
#include <iostream>
#include <cuda.h>
#include <cuda_runtime.h>

// Defining number of elements in Array
#define N 5

// Defining Kernel function for vector addition
__global__ void gpuAdd(int *d_a, int *d_b, int *d_c)
{
    // Getting block index of current kernel
    int tid = blockIdx.x; // handle the data at this index
    if (tid < N)
        d_c[tid] = d_a[tid] + d_b[tid];
}
// Main program
int main(void)
{
    // Defining host arrays
    int h_a[N], h_b[N], h_c[N];
    // Defining device pointers
    int *d_a, *d_b, *d_c;
    // allocate the memory
    cudaMalloc((void**)&d_a, N * sizeof(int));
    cudaMalloc((void**)&d_b, N * sizeof(int));
    cudaMalloc((void**)&d_c, N * sizeof(int));
    // Initializing Arrays
    for (int i = 0; i < N; i++) {
        h_a[i] = 2*i*i;
        h_b[i] = i ;
    }

    // Copy input arrays from host to device memory
    cudaMemcpy(d_a, h_a, N * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, N * sizeof(int), cudaMemcpyHostToDevice);

    // Calling kernels with N blocks and one thread per block, passing
    // device pointers as parameters
    gpuAdd <<<N, 1 >>>(d_a, d_b, d_c);

    // Copy result back to host memory from device memory
    cudaMemcpy(h_c, d_c, N * sizeof(int), cudaMemcpyDeviceToHost);
    printf("Vector addition on GPU \n");

    // Printing result on console
    for (int i = 0; i < N; i++) {
        printf("The sum of %d element is %d + %d = %d\n",
            i, h_a[i], h_b[i],h_c[i]);
    }
    // Free up memory
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    return 0;
}

FileNotFoundError: [Errno 2] No such file or directory: '/usr/local/cuda/bin/nvcc'

# **Matrix Multiplication**

In [ ]:
%%cu
#include <iostream>
#include <cuda.h>

using namespace std;

#define BLOCK_SIZE 2

__global__ void gpuMM(float *A, float *B, float *C, int N)
{
    // Matrix multiplication for NxN matrices C=A*B
    // Each thread computes a single element of C
    int row = blockIdx.y*blockDim.y + threadIdx.y;
    int col = blockIdx.x*blockDim.x + threadIdx.x;

    float sum = 0.f;
    for (int n = 0; n < N; ++n)
        sum += A[row*N+n]*B[n*N+col];

    C[row*N+col] = sum;
}

int main(int argc, char *argv[])
{int N;float K;
    // Perform matrix multiplication C = A*B
    // where A, B and C are NxN matrices
    // Restricted to matrices where N = K*BLOCK_SIZE;
	cout<<"Enter a Value for Size/2 of matrix";
	cin>>K;

    K = 1;
    N = K*BLOCK_SIZE;

    cout << "\n Executing Matrix Multiplcation" << endl;
    cout << "\n Matrix size: " << N << "x" << N << endl;

    // Allocate memory on the host
    float *hA,*hB,*hC;
    hA = new float[N*N];
    hB = new float[N*N];
    hC = new float[N*N];

    // Initialize matrices on the host
    for (int j=0; j<N; j++){
        for (int i=0; i<N; i++){
           hA[j*N+i] = 2;
           hB[j*N+i] = 4;

        }
    }

    // Allocate memory on the device
    int size = N*N*sizeof(float);    // Size of the memory in bytes
    float *dA,*dB,*dC;
    cudaMalloc(&dA,size);
    cudaMalloc(&dB,size);
    cudaMalloc(&dC,size);

    dim3 threadBlock(BLOCK_SIZE,BLOCK_SIZE);
    dim3 grid(K,K);
    cout<<"\n Input Matrix 1 \n";
    for (int row=0; row<N; row++){
            for (int col=0; col<N; col++){

                   cout<<hA[row*col]<<"	";

            }
            cout<<endl;
        }
    cout<<"\n Input Matrix 2 \n";
    for (int row=0; row<N; row++){
            for (int col=0; col<N; col++){

                   cout<<hB[row*col]<<"	";

            }
            cout<<endl;
        }
    // Copy matrices from the host to device
    cudaMemcpy(dA,hA,size,cudaMemcpyHostToDevice);
    cudaMemcpy(dB,hB,size,cudaMemcpyHostToDevice);

    //Execute the matrix multiplication kernel

    gpuMM<<<grid,threadBlock>>>(dA,dB,dC,N);

    // Now do the matrix multiplication on the CPU
   /*float sum;
    for (int row=0; row<N; row++){
        for (int col=0; col<N; col++){
            sum = 0.f;
            for (int n=0; n<N; n++){
                sum += hA[row*N+n]*hB[n*N+col];
            }
            hC[row*N+col] = sum;
            cout << sum <<"	";


        }
        cout<<endl;
    }*/

    // Allocate memory to store the GPU answer on the host
    float *C;
    C = new float[N*N];

    // Now copy the GPU result back to CPU
    cudaMemcpy(C,dC,size,cudaMemcpyDeviceToHost);

    // Check the result and make sure it is correct
    cout <<"\n\n\n\n\n Resultant matrix\n\n";
    for (int row=0; row<N; row++){
        for (int col=0; col<N; col++){

               cout<<C[row*col]<<"	";

        }
        cout<<endl;
    }

    cout << "Finished." << endl;
}